# 从usage中计算总时长和总打开次数（每天）

## 数据准备部分

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from scipy import sparse
import printTime as pt

In [2]:
pt.printTime()

2019-08-20 21:32:41


In [3]:
usage_chunk = pd.read_csv(r'../data/user_app_usage.csv',header=None,chunksize=100000000)
type(usage_chunk)

pandas.io.parsers.TextFileReader

In [4]:
pt.printTime()
date = pd.read_csv(r'../data/user_app_usage.csv',header=None,usecols=[4],nrows=50000).values

2019-08-20 21:32:41


In [5]:
date_dict = {}
i = 0
for one_day in tqdm(date):
    if one_day[0] not in date_dict.keys():
        date_dict[one_day[0]] = i
        i += 1
day_num = i
pt.printTime()

100%|██████████| 50000/50000 [00:00<00:00, 1425693.25it/s]

2019-08-20 21:32:41


In [6]:
preUserId = ''
one_user = np.zeros(2*day_num)
flag = True
usageDuraTimes_Sum = []
for usage in usage_chunk:
    for user in tqdm(usage.values):
        day = date_dict[user[4]]
        currUserId = user[0]
        if flag==True:
            preUserId = currUserId
            flag = False
        if preUserId!=currUserId:
            one_user = np.insert(one_user,0,preUserId)
            usageDuraTimes_Sum.append(one_user)
            one_user = np.zeros(2*day_num)
            preUserId = currUserId
        else:
            one_user[day] += user[2]
            one_user[day+day_num] += user[3]

100%|██████████| 51007719/51007719 [17:57<00:00, 47347.14it/s]   


In [7]:
# usage.sort_values(by=['uId','use_date'],inplace=True)

In [8]:
# preUserId = ''
# one_user = np.zeros(60)
# flag = True
# usageDuraTimes_Sum = []
# for usage in usage_chunk:
#     for user in tqdm(usage.values):
#         stri = user[4].split('-')
#         if int(stri[2])==28 and int(stri[1])==2:
#             day = 0
#         else:
#             day = int(stri[2])
#         currUserId = user[0]
#         if flag==True:
#             preUserId = currUserId
#             flag = False
#         if preUserId!=currUserId:
#             one_user = np.insert(one_user,0,preUserId)
#             usageDuraTimes_Sum.append(one_user)
#             one_user = np.zeros(60)
#             preUserId = currUserId
#         else:
#             one_user[day] += user[2]
#             one_user[day+30] += user[3]

In [9]:
one_user = np.insert(one_user,0,preUserId)
usageDuraTimes_Sum.append(one_user)
pt.printTime()

2019-08-20 22:31:10


In [10]:
usageDuraTimes_Sum = pd.DataFrame(usageDuraTimes_Sum)
usageDuraTimes_Sum.shape

(1992667, 61)

In [11]:
# usageDuraTimes_Sum.head()

In [12]:
pt.printTime()
usageDuraTimes_Sum.to_csv(r'../processed/usageDuraTimes_Sum.csv',header=None,index=False)
pt.printTime()

2019-08-20 22:34:07
2019-08-20 22:36:08


## 数据处理部分，生成trainData60.npz和testData60.npz文件

In [13]:
# usageDuraTimes_Sum = pd.read_csv(r'../processed/usageDuraTimes_Sum.csv',header=None)
# print(usageDuraTimes_Sum.shape)
# print(usageDuraTimes_Sum.head())

In [14]:
pt.printTime()
usageDuraTimes_Sum = usageDuraTimes_Sum.values

2019-08-20 22:36:08


In [15]:
usageDuraTimes_Sum[:,1:] = preprocessing.StandardScaler().fit_transform(usageDuraTimes_Sum[:,1:])

In [16]:
usageDuraTimes_Sum = pd.DataFrame(usageDuraTimes_Sum)
print(usageDuraTimes_Sum.shape)
print(usageDuraTimes_Sum.head())

(1992667, 61)
          0         1         2         3         4         5         6   \
0  1000364.0 -0.736996 -0.643930 -0.572220 -0.525964 -0.972846 -0.493046   
1  1000387.0  1.246786  0.678483 -0.040560  1.835045  0.989643  1.526530   
2  1000872.0 -0.414773 -0.571459 -0.481216 -0.417523 -0.581154 -0.498701   
3  1000951.0 -1.074658 -0.298232 -0.223638  0.064338 -0.941143 -0.630345   
4  1001091.0  0.223019 -0.150401  0.380451  0.204801  0.764011  1.023954   

         7         8         9   ...        51        52        53        54  \
0 -0.624513 -0.652973 -0.589532  ... -0.712512 -0.506526 -0.656138 -0.526741   
1  0.429499  0.958169  1.324566  ...  1.102472  1.200109  0.090805  0.329822   
2 -0.440690 -0.393840 -0.587664  ...  0.697955 -0.180075  0.137575 -0.173175   
3 -0.233799 -0.097233 -0.776698  ... -0.668244 -0.512035 -0.438795 -0.596120   
4  2.189435  0.167681  1.022279  ...  0.102628  1.441159 -0.045378  1.317138   

         55        56        57        58       

In [17]:
age_train = pd.read_csv(r'../data/age_train.csv',header=None,usecols=[0])
age_test = pd.read_csv(r'../data/age_test.csv',header=None)

In [18]:
imp = SimpleImputer()

In [19]:
train = pd.merge(age_train,usageDuraTimes_Sum,how='inner',on=0)
train = pd.merge(age_train,train,how='outer',on=0)

In [20]:
train = sparse.csr_matrix(imp.fit_transform(train.values[:,1:]))
train.shape

(2010000, 60)

In [21]:
test = pd.merge(age_test,usageDuraTimes_Sum,how='inner',on=0)
test = pd.merge(age_test,test,how='outer',on=0)

In [22]:
test = sparse.csr_matrix(imp.fit_transform(test.values[:,1:]))
test.shape

(502500, 60)

In [23]:
sparse.save_npz(r'../trainTestData/trainData60.npz',train)
sparse.save_npz(r'../trainTestData/testData60.npz',test)
pt.printTime()

2019-08-20 22:40:11
